In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,3692
2,Huelva,Confirmados PDIA 14 días,1586
3,Huelva,Tasa PDIA 14 días,"310,52799548892494"
4,Huelva,Confirmados PDIA 7 días,938
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,26
629,Municipio de Huelva sin especificar,Total Confirmados,58
630,Municipio de Huelva sin especificar,Curados,11


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  3692.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  976.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 557 personas en los últimos 7 días 

Un positivo PCR cada 349 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,3692.0,938.0,1586.0,510743.0,183.654010,310.527995,165.0
Huelva-Costa,1857.0,449.0,766.0,288115.0,155.840550,265.866060,76.0
Condado-Campiña,1174.0,290.0,457.0,155057.0,187.027996,294.730325,61.0
Huelva (capital),976.0,258.0,412.0,143663.0,179.586950,286.782261,40.0
Sierra de Huelva-Andévalo Central,603.0,173.0,336.0,67571.0,256.026994,497.254739,28.0
Almonte,185.0,73.0,115.0,24191.0,301.765119,475.383407,15.0
Palma del Condado (La),173.0,40.0,67.0,10761.0,371.712666,622.618716,9.0
Valverde del Camino,110.0,54.0,76.0,12820.0,421.216849,592.823713,8.0
Aracena,98.0,27.0,38.0,8107.0,333.045516,468.730727,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granado (El),34.0,3.0,19.0,516.0,581.395349,3682.170543,1.0
Cortegana,111.0,26.0,95.0,4666.0,557.222460,2036.005144,0.0
Escacena del Campo,46.0,24.0,27.0,2284.0,1050.788091,1182.136602,4.0
Villarrasa,53.0,19.0,24.0,2176.0,873.161765,1102.941176,4.0
Almonaster la Real,20.0,8.0,18.0,1819.0,439.802089,989.554700,1.0
Cerro de Andévalo (El),25.0,3.0,17.0,2364.0,126.903553,719.120135,0.0
Calañas,23.0,18.0,20.0,2791.0,644.930133,716.589036,0.0
Palma del Condado (La),173.0,40.0,67.0,10761.0,371.712666,622.618716,9.0
Valverde del Camino,110.0,54.0,76.0,12820.0,421.216849,592.823713,8.0
